# भाग 8 बीआईएस - Protocols (प्रोटोकॉल) का परिचय

### प्रसंग

अब जब हम प्‍लान के माध्यम से आ गए हैं, तो हम प्रोटोकॉल नामक एक नई वस्तु पेश करेंगे। एक प्रोटोकॉल प्‍लान के अनुक्रम का समन्वय करता है, उन्हें दूर के श्रमिकों पर तैनात करता है और उन्हें एक पास में चलाता है।

यह एक उच्च स्तरीय वस्तु है जिसमें एक जटिल संगणना का तर्क समाहित है कई श्रमिकों में वितरित किया गया। प्रोटोकॉल की मुख्य विशेषता है
श्रमिकों के बीच वापस भेजने / खोजे जाने / पाने की क्षमता और अंत में चिन्हित श्रमिकों को तैनात किया गया। तो एक उपयोगकर्ता एक प्रोटोकॉल डिजाइन कर सकता है, उसे क्लाउड वर्कर  में अपलोड कर सकता है, और कोई भी अन्य कर्मी इसकी खोज करने में सक्षम होगा, इसे डाउनलोड कर सकता है, और श्रमिकों जिस पर यह जुड़ा हुआ है उन पर यह गणना कार्यक्रम लागू कर सकता है।

आइए देखें कि इसका उपयोग कैसे करें!

लेखक:
- Théo Ryffel - Twitter [@theoryffel](https://twitter.com/theoryffel) - Github: [@LaRiffle](https://github.com/LaRiffle)

nbTranslate का उपयोग करके अनुवादित

संपादक:

- Urvashi Raheja - Github: [@raheja](https://github.com/raheja)


### 1. बनाएं और तैनात करें

प्रोटोकॉल जोड़े `(worker, plan)` की एक सूची प्रदान करके बनाए गए हैं। `worker` या तो एक वास्तविक कार्यकर्ता हो सकता है या कार्यकर्ता आईडी या एक स्ट्रिंग एक काल्पनिक कार्यकर्ता का प्रतिनिधित्व करने के लिए। यह अंतिम मामला, निर्माण में इस्तेमाल किया जा सकता है, यह निर्दिष्ट करने के लिए कि दो प्‍लान तैनाती पर एक ही कार्यकर्ता द्वारा स्वामित्व (या स्वामित्व में नहीं) होने चाहिए। `plan` या तो एक प्‍लान हो या एक PointerPlan हो।

In [ ]:
import torch as th
import syft as sy
hook = sy.TorchHook(th)

# IMPORTANT: Local worker should not be a client worker
hook.local_worker.is_client_worker = False

आइए 3 प्‍लान को परिभाषित करें और उन्हें एक प्रोटोकॉल में भेज दे। वे सभी एक क्रमशः वृद्धि ऑपरेशन करते हैं।

In [ ]:
@sy.func2plan(args_shape=[(1,)])
def inc1(x):
    return x + 1

@sy.func2plan(args_shape=[(1,)])
def inc2(x):
    return x + 1

@sy.func2plan(args_shape=[(1,)])
def inc3(x):
    return x + 1

protocol = sy.Protocol([("worker1", inc1), ("worker2", inc2), ("worker3", inc3)])

अब हमें श्रमिकों को प्रोटोकॉल बाँधने की आवश्यकता है, जिसे `.deploy (*workers)` कहकर किया जाता है। चलो कुछ कार्यकर्ता बनाते हैं।

In [ ]:
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
charlie = sy.VirtualWorker(hook, id="charlie")

In [ ]:
workers = alice, bob, charlie

protocol.deploy(*workers)

आप देख सकते हैं कि प्‍लान पहले ही उपयुक्त श्रमिकों को भेज दी गई हैं: इसे तैनात किया गया है!

यह 2 चरणों में किया गया है: पहला, हम निर्माण में प्रदान किए गए काल्पनिक श्रमिकों का नक्शा बनाते हैं
(नाम स्ट्रिंग्स द्वारा) प्रदान किए गए श्रमिकों के लिए, और दूसरा, हम संबंधित को उनमें से प्रत्येक के लिए प्‍लान भेजते हैं। 

### 2. एक प्रोटोकॉल चलाएँ

एक प्रोटोकॉल चलाने का मतलब है कि सभी प्‍लान को क्रमिक रूप से निष्पादित करना। ऐसा करने के लिए, आप कुछ इनपुट डेटा प्रदान करते हैं जो पहले प्‍लान स्थान पर भेजा जाता है। यह प्रथम प्‍लान को रन और उसके आउटपुट को दूसरी प्‍लान स्थान पर ले जाया जाता है, और इसी तरह दोहराया जाता है। अंतिम
सभी प्‍लान के चलने के बाद परिणाम लौटाया जाता है, और यह अंतिम प्‍लान स्थान के पॉइंटर से बना होता है।

In [ ]:
x = th.tensor([1.0])
ptr = protocol.run(x)
ptr

In [ ]:
ptr.get()

1.0 इनपुट 3 प्‍लान के माध्यम से है और इसलिए 3 बार वृद्धि की गई है, यही कारण है कि अब यह 4.0 के बराबर है!

वास्तव में, आप डेटा पर कुछ पॉइंटर से **एक प्रोटोकॉल दूरस्थ रूप** में चला सकते हैं:

In [ ]:
james = sy.VirtualWorker(hook, id="james")

In [ ]:
protocol.send(james)

In [ ]:
x = th.tensor([1.0]).send(james)
ptr = protocol.run(x)
ptr

जैसा कि आप देखते हैं कि परिणाम जेमर्स के लिए एक सूचक है

In [ ]:
ptr = ptr.get()
ptr

In [ ]:
ptr = ptr.get()
ptr

### 3. एक प्रोटोकॉल के लिए खोजें

वास्तविक सेटिंग्स में आप एक दूरस्थ प्रोटोकॉल डाउनलोड करना चाहते हैं, इसे अपने कार्यकर्ताओं पर तैनात कर सकते हैं और इसे आपके डेटा के साथ चला सकते हैं:

आइए एक प्रोटोकॉल को शुरू करें **जिसे परिनियोजित नहीं किया गया है**, और इसे एक दूरस्थ कार्यकर्ता पर रखें

In [ ]:
protocol = sy.Protocol([("worker1", inc1), ("worker2", inc2), ("worker3", inc3)])
protocol.tag('my_protocol')
protocol.send(james)

In [ ]:
me = sy.hook.local_worker # get access to me as a local worker

अब हम प्रोटोकॉल खोजने के लिए एक खोज शुरू करते हैं

In [ ]:
responses = me.request_search(['my_protocol'], location=james)
responses

आपके पास एक प्रोटोकॉल के लिए एक पॉइंटर तक पहुंच है

In [ ]:
ptr_protocol = responses[0]

सामान्य सूचक की तरह आप इसे वापस पा सकते हैं:

In [ ]:
protocol_back = ptr_protocol.get()
protocol_back

और हम कर सकते हैं जैसे हमने भागों 1 और 2 में किया था।

In [ ]:
protocol_back.deploy(alice, bob, charlie)

x = th.tensor([1.0])
ptr = protocol_back.run(x)
ptr.get()

अधिक वास्तविक दुनिया के उदाहरण प्रोटोकॉल के साथ आएंगे, लेकिन आप पहले से ही इस नई वस्तु द्वारा खोली गई सभी संभावनाओं को देख सकते हैं!

### GitHub पर स्टार PySyft

हमारे समुदाय की मदद करने का सबसे आसान तरीका सिर्फ रिपॉजिटरी को अभिनीत करना है! यह हमारे द्वारा बनाए जा रहे कूल टूल्स के बारे में जागरूकता बढ़ाने में मदद करता है।

- [स्टार पायसफ्ट](https://github.com/OpenMined/PySyft)

### GitHub पर हमारे ट्यूटोरियल चुनना!

हमने फेडरेटेड और प्राइवेसी-प्रिजर्विंग लर्निंग और उसकी संरचना की बेहतर समझ प्राप्त करने के लिए वास्तव में अच्छा ट्यूटोरियल बनाया है।

- [PySyft ट्यूटोरियल्स को चेक करें](https://github.com/OpenMined/PySyft/tree/master/examples/tutorials)


### हमारे Slack में शामिल हों!

नवीनतम प्रगति पर अद्यतित रहने का सबसे अच्छा तरीका हमारे समुदाय में शामिल होना है!

- [slack.openmined.org से जुड़ें](http://slack.openmined.org)

### एक कोड परियोजना में शामिल हों!

हमारे समुदाय में योगदान करने का सबसे अच्छा तरीका एक कोड योगदानकर्ता बनना है! यदि आप "one off" मिनी-प्रोजेक्ट्स शुरू करना चाहते हैं, तो आप PySyft GitHub जारी करने वाले पृष्ठ पर जा सकते हैं और `Good First Issue` के रूप में चिह्नित मुद्दों की खोज कर सकते हैं।

- [Good First Issue Tickets](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22)

### दान करना

यदि आपके पास हमारे कोडबेस में योगदान करने का समय नहीं है, लेकिन फिर भी समर्थन उधार देना चाहते हैं, तो आप हमारे ओपन कलेक्टिव में भी एक बैकर बन सकते हैं। सभी दान हमारी वेब होस्टिंग और अन्य सामुदायिक खर्च जैसे कि हैकाथॉन और मीटअप की ओर जाते हैं!

- [OpenMined's Open कलेक्टिव पेज के माध्यम से दान करें](https://opencollective.com/openmined)